In [62]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import set_config
import os
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
import numpy as np
from sqlalchemy import create_engine
from DealMatch.database_connector import db_connector
from dotenv import load_dotenv
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
os.getcwd()

'/Users/michaelbroza/code/michabrz/DealMatch/notebooks'

In [3]:
pd.set_option('display.max_columns', 500)

In [64]:
dbConnection = db_connector()

OperationalError: (pymysql.err.OperationalError) (1045, "Access denied for user 'readonly_user'@'62.96.4.194' (using password: YES)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

# Read in the Target Data

In [55]:
query = """
    SELECT 
        d.id as deal_id,
        d.name as deal_name,
        dt.name as deal_type_name,
        d.target_company_id,
        d.target_name,
        d.target_description,
        d.target_revenue,
        d.target_ebitda,
        d.target_ebit,
        c2.name_en as country_name,
        r.name_en as region_name,
        s.name_en as sector_name,
        s2.name_en as subsector_name,
        k.name_de as keyword
    FROM deals d 
    LEFT JOIN deal_stages ds ON ds.id = d.deal_stage_id 
    LEFT JOIN deal_types dt ON dt.id = d.deal_type_id 
    LEFT JOIN deal_lost_reasons dlr ON dlr.id = d.lost_reason_id 
    LEFT JOIN countries c2 ON d.target_country_id = c2.id 
    LEFT JOIN regions r ON d.target_region_id = r.id 
    JOIN deal_sectors ds2 ON ds2.deal_id = d.id 
    JOIN sectors s ON s.id = ds2.sector_id 
    JOIN deal_subsectors ds3 ON ds2.id = ds3.deal_sector_id 
    JOIN subsectors s2 ON s2.id = ds3.subsector_id 
    JOIN deal_keywords dk ON dk.deal_id = d.id
    JOIN keywords k ON dk.keyword_id = k.id 
"""

In [56]:
targets = pd.read_sql(query, dbConnection)
targets.head()

OperationalError: (pymysql.err.OperationalError) (1045, "Access denied for user 'readonly_user'@'62.96.4.194' (using password: YES)")
[SQL: 
    SELECT 
        d.id as deal_id,
        d.name as deal_name,
        dt.name as deal_type_name,
        d.target_company_id,
        d.target_name,
        d.target_description,
        d.target_revenue,
        d.target_ebitda,
        d.target_ebit,
        c2.name_en as country_name,
        r.name_en as region_name,
        s.name_en as sector_name,
        s2.name_en as subsector_name,
        k.name_de as keyword
    FROM deals d 
    LEFT JOIN deal_stages ds ON ds.id = d.deal_stage_id 
    LEFT JOIN deal_types dt ON dt.id = d.deal_type_id 
    LEFT JOIN deal_lost_reasons dlr ON dlr.id = d.lost_reason_id 
    LEFT JOIN countries c2 ON d.target_country_id = c2.id 
    LEFT JOIN regions r ON d.target_region_id = r.id 
    JOIN deal_sectors ds2 ON ds2.deal_id = d.id 
    JOIN sectors s ON s.id = ds2.sector_id 
    JOIN deal_subsectors ds3 

In [57]:
targets.shape

(348066, 13)

In [ ]:
targets.to_excel('targets_raw.xlsx', encoding="UTF-8")

In [52]:
targets_clean.to_excel('targets_clean.xlsx', encoding="UTF-8")

In [10]:
targets['deal_id'] = targets.groupby(['deal_name']).ngroup()

In [11]:
targets['target_company_id'] = targets.groupby(['target_name']).ngroup()

In [12]:
targets.head()

,deal_id,deal_name,deal_type_name,target_company_id,target_name,target_description,target_revenue,target_ebitda,target_ebit,country_name,region_name,sector_name,subsector_name,keyword
0,173,1301 Sun,OTHER,871,Sun [Target],•\t350MWp of solar photovoltaic project assets...,NaN,NaN,NaN,None,None,Energy,Solar power,solar
1,173,1301 Sun,OTHER,871,Sun [Target],•\t350MWp of solar photovoltaic project assets...,NaN,NaN,NaN,None,None,Energy,Solar power,pv
2,173,1301 Sun,OTHER,871,Sun [Target],•\t350MWp of solar photovoltaic project assets...,NaN,NaN,NaN,None,None,Energy,Solar power,energie
3,173,1301 Sun,OTHER,871,Sun [Target],•\t350MWp of solar photovoltaic project assets...,NaN,NaN,NaN,None,None,Energy,Solar power,photovoltaik
4,129,1220 Supple,OTHER,874,Supple [Target],Solaranlage in Kreta,57.7,NaN,NaN,None,None,Energy,Solar power,solar


In [13]:
g = targets.groupby('deal_id')['sector_name', 'subsector_name', 'keyword'].apply(lambda x: list(np.unique(x)))

/var/folders/n8/1rj47qrj1zbgzvm5m8slt_lc0000gn/T/ipykernel_15824/4265419136.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  g = targets.groupby('deal_id')['sector_name', 'subsector_name', 'keyword'].apply(lambda x: list(np.unique(x)))


In [14]:
g = targets.groupby('deal_id')['sector_name', 'subsector_name',
                               'keyword'].apply(lambda x: list(np.unique(x)))
g = pd.DataFrame(g, columns=['keywords_all'])
g['liststring'] = [', '.join(map(str, l)) for l in g['keywords_all']]
g.drop(columns="keywords_all", inplace=True)
g.reset_index(inplace=True)

In [15]:
targets = pd.merge(targets, g, on="deal_id", how="left")

In [16]:
targets.drop(columns=['subsector_name', 'keyword'], inplace=True)

In [17]:
targets.columns

Index(['deal_id', 'deal_name', 'deal_type_name', 'target_company_id',
       'target_name', 'target_description', 'target_revenue', 'target_ebitda',
       'target_ebit', 'country_name', 'region_name', 'sector_name',
       'liststring'],
      dtype='object')

In [18]:
targets_clean = targets.drop_duplicates(subset="deal_id")

In [19]:
targets_clean.shape

(1111, 13)

In [20]:
targets_clean.head()

,deal_id,deal_name,deal_type_name,target_company_id,target_name,target_description,target_revenue,target_ebitda,target_ebit,country_name,region_name,sector_name,liststring
0,173,1301 Sun,OTHER,871,Sun [Target],•\t350MWp of solar photovoltaic project assets...,NaN,NaN,NaN,None,None,Energy,"Energy, Solar power, energie, photovoltaik, pv..."
4,129,1220 Supple,OTHER,874,Supple [Target],Solaranlage in Kreta,57.70,NaN,NaN,None,None,Energy,"Energy, Renewable energy - other, Solar power,..."
16,407,1677 Heat,MAJORITY,806,SPH Sustainable Process Heat GmbH,PROJECT HEAT hat eine neue Wärmepumpentechnolo...,0.43,-0.78,-0.78,None,None,Industrial products and services,"Industrial products and services, Pumps and co..."
22,1013,845 Apollo,OTHER,812,SUMMIQ AG,Fundraising für Renewable Holding,0.00,0.00,0.00,Germany,Bavaria,Financial Services,"Financial Services, Other Diversified Financia..."
26,752,390 Saragossa,OTHER,507,KSW Bioenergie GmbH,Errichtung eines CO2-neutralen Bio-Energie Kra...,0.00,0.00,0.00,Germany,North Rhine-Westphalia,Energy,"Bioenergy, Energy, bioenergie, biomasse, biotr..."


In [30]:
# Add it at the end of our numerical transformer
num_transformer = Pipeline([('imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)),
                            ('scaler', RobustScaler())])

cat_transformer = Pipeline([('imputer',
                             SimpleImputer(missing_values=np.nan,
                                           strategy='constant',
                                           fill_value='no_region')),
                            ('ohe',
                             OneHotEncoder(handle_unknown='ignore',
                                           sparse=False))])

preprocessor = ColumnTransformer([
    ('num_tr', num_transformer,
     ['target_revenue', 'target_ebitda', 'target_ebit']),
    ('cat_tr', cat_transformer, ['deal_type_name', 'country_name', 'region_name', 'sector_name'])
],
                                 remainder='passthrough')

#pipeline = Pipeline([('preprocessor', preprocessor), 'LogisticRegression',
#                     MLPClassifier(alpha=1, max_iter=1000])


In [31]:
from sklearn import set_config

set_config(display='diagram')
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('num_tr',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant')),
                                                 ('scaler', RobustScaler())]),
                                 ['target_revenue', 'target_ebitda',
                                  'target_ebit']),
                                ('cat_tr',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='no_region',
                                                                strategy='constant')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['deal_type_name', 'country_name',
                                  'region_name', 'sector_name'])])

In [32]:
data = targets_clean.copy()

In [33]:
data_transformed = preprocessor.fit_transform(data)

display(data.head(3))
display(pd.DataFrame(data_transformed).head(3))

,deal_id,deal_name,deal_type_name,target_company_id,target_name,target_description,target_revenue,target_ebitda,target_ebit,country_name,region_name,sector_name,liststring
0,173,1301 Sun,OTHER,871,Sun [Target],•\t350MWp of solar photovoltaic project assets...,NaN,NaN,NaN,None,None,Energy,"Energy, Solar power, energie, photovoltaik, pv..."
4,129,1220 Supple,OTHER,874,Supple [Target],Solaranlage in Kreta,57.70,NaN,NaN,None,None,Energy,"Energy, Renewable energy - other, Solar power,..."
16,407,1677 Heat,MAJORITY,806,SPH Sustainable Process Heat GmbH,PROJECT HEAT hat eine neue Wärmepumpentechnolo...,0.43,-0.78,-0.78,None,None,Industrial products and services,"Industrial products and services, Pumps and co..."


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80
0,-0.603689,-0.46729,-0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,173,1301 Sun,871,Sun [Target],•\t350MWp of solar photovoltaic project assets...,"Energy, Solar power, energie, photovoltaik, pv..."
1,5.846842,-0.46729,-0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,129,1220 Supple,874,Supple [Target],Solaranlage in Kreta,"Energy, Renewable energy - other, Solar power,..."
2,-0.555618,-1.196262,-1.28,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,407,1677 Heat,806,SPH Sustainable Process Heat GmbH,PROJECT HEAT hat eine neue Wärmepumpentechnolo...,"Industrial products and services, Pumps and co..."


In [35]:
SimpleImputer.get_feature_names_out = (
    lambda self, names=None: self.feature_names_in_)

In [36]:
preprocessor.get_feature_names_out()

array(['num_tr__target_revenue', 'num_tr__target_ebitda',
       'num_tr__target_ebit', 'cat_tr__deal_type_name_DISTRESSED',
       'cat_tr__deal_type_name_MAJORITY',
       'cat_tr__deal_type_name_MINORITY', 'cat_tr__deal_type_name_OTHER',
       'cat_tr__deal_type_name_VC', 'cat_tr__deal_type_name_None',
       'cat_tr__country_name_Austria', 'cat_tr__country_name_Belgium',
       'cat_tr__country_name_Bulgaria', 'cat_tr__country_name_Czechia',
       'cat_tr__country_name_Egypt', 'cat_tr__country_name_Germany',
       'cat_tr__country_name_Italy', 'cat_tr__country_name_Netherlands',
       'cat_tr__country_name_Poland', 'cat_tr__country_name_Portugal',
       'cat_tr__country_name_Romania', 'cat_tr__country_name_Slovakia',
       'cat_tr__country_name_Spain', 'cat_tr__country_name_Switzerland',
       'cat_tr__country_name_United States of America',
       'cat_tr__country_name_None',
       'cat_tr__region_name_Baden-Württemberg',
       'cat_tr__region_name_Bavaria', 'cat_tr__regi

In [44]:
data_transformed = pd.DataFrame(data_transformed,
             columns=preprocessor.get_feature_names_out())

In [47]:
data_transformed.head()

,num_tr__target_revenue,num_tr__target_ebitda,num_tr__target_ebit,cat_tr__deal_type_name_DISTRESSED,cat_tr__deal_type_name_MAJORITY,cat_tr__deal_type_name_MINORITY,cat_tr__deal_type_name_OTHER,cat_tr__deal_type_name_VC,cat_tr__deal_type_name_None,cat_tr__country_name_Austria,cat_tr__country_name_Belgium,cat_tr__country_name_Bulgaria,cat_tr__country_name_Czechia,cat_tr__country_name_Egypt,cat_tr__country_name_Germany,cat_tr__country_name_Italy,cat_tr__country_name_Netherlands,cat_tr__country_name_Poland,cat_tr__country_name_Portugal,cat_tr__country_name_Romania,cat_tr__country_name_Slovakia,cat_tr__country_name_Spain,cat_tr__country_name_Switzerland,cat_tr__country_name_United States of America,cat_tr__country_name_None,cat_tr__region_name_Baden-Württemberg,cat_tr__region_name_Bavaria,cat_tr__region_name_Berlin,cat_tr__region_name_Brandenburg,cat_tr__region_name_Bremen,cat_tr__region_name_Hamburg,cat_tr__region_name_Hesse,cat_tr__region_name_Lower Saxony,cat_tr__region_name_Mecklenburg-Vorpommern,cat_tr__region_name_North Rhine-Westphalia,cat_tr__region_name_Rhineland-Palatinate,cat_tr__region_name_Saarland,cat_tr__region_name_Saxony,cat_tr__region_name_Saxony-Anhalt,cat_tr__region_name_Schleswig-Holstein,cat_tr__region_name_Thuringia,cat_tr__region_name_None,cat_tr__sector_name_Agriculture,cat_tr__sector_name_Automotive,cat_tr__sector_name_Biotechnology & Life Sciences,cat_tr__sector_name_Chemicals,cat_tr__sector_name_Computer Hardware & Equipment,cat_tr__sector_name_Construction,cat_tr__sector_name_Consumer Goods & Apparel,cat_tr__sector_name_Defense,cat_tr__sector_name_Electronics,cat_tr__sector_name_Energy,cat_tr__sector_name_Financial Services,cat_tr__sector_name_Food & Beverages,cat_tr__sector_name_Food & Staples Retailing,cat_tr__sector_name_Health Care Equipment & Services,cat_tr__sector_name_IT services,cat_tr__sector_name_Industrial automation,cat_tr__sector_name_Industrial products and services,cat_tr__sector_name_Insurance,cat_tr__sector_name_Internet/ecommerce,cat_tr__sector_name_Leisure & consumer services,cat_tr__sector_name_Manufacturing (other),cat_tr__sector_name_Media,cat_tr__sector_name_Mining,cat_tr__sector_name_Pharmaceuticals,cat_tr__sector_name_Professional Services (B2B),cat_tr__sector_name_Real Estate,cat_tr__sector_name_Retailing,cat_tr__sector_name_Semiconductors & Semiconductor Equipment,cat_tr__sector_name_Software & Services,cat_tr__sector_name_Telecommunication Hardware,cat_tr__sector_name_Telecommunication Services,cat_tr__sector_name_Transportation,cat_tr__sector_name_Utilities,remainder__deal_id,remainder__deal_name,remainder__target_company_id,remainder__target_name,remainder__target_description,remainder__liststring
0,-0.603689,-0.46729,-0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,173,1301 Sun,871,Sun [Target],•\t350MWp of solar photovoltaic project assets...,"Energy, Solar power, energie, photovoltaik, pv..."
1,5.846842,-0.46729,-0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,129,1220 Supple,874,Supple [Target],Solaranlage in Kreta,"Energy, Renewable energy - other, Solar power,..."
2,-0.555618,-1.196262,-1.28,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,407,1677 Heat,806,SPH Sustainable Process Heat GmbH,PROJECT HEAT hat eine neue Wärmepumpentechnolo...,"Industrial pr

In [53]:
data_transformed.columns

Index(['num_tr__target_revenue', 'num_tr__target_ebitda',
       'num_tr__target_ebit', 'cat_tr__deal_type_name_DISTRESSED',
       'cat_tr__deal_type_name_MAJORITY', 'cat_tr__deal_type_name_MINORITY',
       'cat_tr__deal_type_name_OTHER', 'cat_tr__deal_type_name_VC',
       'cat_tr__deal_type_name_None', 'cat_tr__country_name_Austria',
       'cat_tr__country_name_Belgium', 'cat_tr__country_name_Bulgaria',
       'cat_tr__country_name_Czechia', 'cat_tr__country_name_Egypt',
       'cat_tr__country_name_Germany', 'cat_tr__country_name_Italy',
       'cat_tr__country_name_Netherlands', 'cat_tr__country_name_Poland',
       'cat_tr__country_name_Portugal', 'cat_tr__country_name_Romania',
       'cat_tr__country_name_Slovakia', 'cat_tr__country_name_Spain',
       'cat_tr__country_name_Switzerland',
       'cat_tr__country_name_United States of America',
       'cat_tr__country_name_None', 'cat_tr__region_name_Baden-Württemberg',
       'cat_tr__region_name_Bavaria', 'cat_tr__region_name